In [48]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

data = load_iris()

In [49]:
X = data.data
y = data.target

In [50]:
y = pd.get_dummies(y)

In [51]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [53]:
class Dense():
    def __init__(self,size,input_shape):
        self.size = size
        self.input_shape = input_shape
        self.weights = np.random.random([self.size,self.input_shape])
        self.biases = np.random.random([self.size,1])
        self.dL = np.zeros(size)

    def sigmoid(self,X):
        X_normalized = X - X.mean()  
        return 1 / (1 + np.exp(-X_normalized))

    # def relu(self,X):
    #     return np.max(X,0)

    def forward(self,inputs):
        self.input = inputs
        self.output = np.dot(self.weights,inputs) + self.biases
        self.output = self.sigmoid(self.output)
        return self.output
    
    def update(self,learning_rate):
        # print(self.biases.shape)
        # print(self.dL.reshape(-1,1).shape)
        self.biases -= learning_rate*(self.dL.reshape(-1,1))
        # print(self.input.mean(axis=1).shape)
        # print(self.dL.shape)
        temp1 = self.input.mean(axis=1).reshape(1,-1)
        temp2 = self.dL.reshape(1,-1)
        self.dW = np.dot(temp1.T, temp2)
        self.dW = self.dW.reshape(self.size,-1)
        # print(self.dW.shape)
        # print(self.weights.shape)
        self.weights -= learning_rate*self.dW

In [54]:
class NeuralNetwork():
    def __init__(self,layers):
        self.layers = layers

    def fit(self,X,y,epochs):
        self.m = X.shape[0]
        y = np.array(y).transpose()
        for i in range(epochs):
            #forward pass
            input = np.array(X).transpose()
            for layer in self.layers:
                input = layer.forward(input)
            y_hat = input
    
            #calculate loss
            if((i+1)%100==0):
                print("Epoch: ",i+1,"Error: ", self.mse(y_hat,y))
            
            #backward pass
            loss = self.mse(y_hat,y)
            layer_count = 1
    
            
            for layer in reversed(self.layers):
                #calcuate 
                if(layer_count==1):
                    dL = ((layer.output - y) * (layer.output*(1-layer.output))).mean(axis=1)
                    layer.dL = dL
                    # print(layer.dL, "shape", layer.dL.shape)
                    temp = np.dot(layer.weights.T, dL)
                else:
                    dL = (temp * (layer.output*(1-layer.output)).T).mean(axis=0)
                    layer.dL = dL
                    # print(layer.dL, "shape", layer.dL.shape)
                    temp = np.dot(layer.weights.T, dL)
                    
                layer_count+=1
                layer.update(0.00001)

    
    def predict(self,X):
        input = np.array(X).transpose()
        for layer in self.layers:
            input = layer.forward(input)
        return input.T

    def score(self,X):
        pass

    def mse(self,y_hat,y):
        # print(y_hat.shape)
        # print((y_hat - y).sum(axis=1).shape)
        squared_errors = (1/self.m) * (y_hat - y).mean(axis=0).sum() ** 2
        return np.mean(squared_errors)

In [62]:
layer1 = Dense(size=12,input_shape=4)
layer2 = Dense(size=10,input_shape=12)
layer3 = Dense(size=3,input_shape=10)

model = NeuralNetwork([layer1,layer2,layer3])

model.fit(X_train,y_train,epochs=20000)

Epoch:  100 Error:  2.9146459075790525
Epoch:  200 Error:  2.9146456866384063
Epoch:  300 Error:  2.914645465445896
Epoch:  400 Error:  2.9146452440016177
Epoch:  500 Error:  2.914645022305673
Epoch:  600 Error:  2.9146448003581504
Epoch:  700 Error:  2.9146445781591495
Epoch:  800 Error:  2.9146443557087713
Epoch:  900 Error:  2.9146441330070987
Epoch:  1000 Error:  2.914643910054238
Epoch:  1100 Error:  2.9146436868502823
Epoch:  1200 Error:  2.914643463395332
Epoch:  1300 Error:  2.914643239689472
Epoch:  1400 Error:  2.9146430157328083
Epoch:  1500 Error:  2.9146427915254307
Epoch:  1600 Error:  2.9146425670674345
Epoch:  1700 Error:  2.914642342358922
Epoch:  1800 Error:  2.9146421173999864
Epoch:  1900 Error:  2.914641892190719
Epoch:  2000 Error:  2.9146416667312223
Epoch:  2100 Error:  2.914641441021582
Epoch:  2200 Error:  2.914641215061904
Epoch:  2300 Error:  2.9146409888522786
Epoch:  2400 Error:  2.9146407623928052
Epoch:  2500 Error:  2.9146405356835796
Epoch:  2600 Error

In [63]:
y_pred = model.predict(X_test)

In [64]:
y_pred.shape

(45, 3)

In [65]:
y_pred.argmax(axis=1)

array([1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)

In [59]:
y_test.values.argmax(axis=1)

array([2, 2, 1, 1, 2, 0, 1, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 1, 2, 1, 1, 2,
       1, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 0, 1,
       2], dtype=int64)